# IMPORTS

In [1]:
import json
import pandas as pd
import subprocess as SB
import xml.etree.ElementTree as ET

# HELPERS

In [2]:
def getOrientation(shap_json):
    empty = {}
    for el in shap_json:
        term = el['term']
        orientation = el['orientation']
        empty[term] = orientation
    return empty

In [3]:
def getIDs(shap_json):
    empty = {}
    for el in shap_json:
        term = el['term']
        identifier = el['id']
        empty[term] = identifier
    return empty

In [4]:
def getFactors(data):
    factors = {}
    for el in data:
        term = el["term"]
        factor = el["factor"]
        factors[term] = factor
    return factors

# GENERATE INPUT FILE  cloud.jar

In [5]:
def createInputFile(factors, filename):
    file = filename + ".txt"
    words = ""
    for key in factors:
        v = factors[key]
        size = (float(v) * 100)    
        for i in range(1, int(size)):
            words += key + " "
        words += ". "

    with open(file, "w") as text_file:
        text_file.write(words)

# GENERATE SVG

In [6]:
def generateSVG(filename):
    file = filename + ".txt"
    args = ['java', '-jar', 'cloudy.jar', '-w1200', '-h500', '-Lmds', '-ps', '-pg', '-pn', '-O', file]
    SB.call(args)

# PARSE SVG

In [7]:
def parseSVG(filename, ids, orientations):
    file = filename + ".svg"
    root = ET.parse(file).getroot()
    internal = root[1]
    result = '<svg width="550" height="300" font-family="Arial" fill="#a9a9a9" font-style="normal" font-weight="bold"> <g transform="scale(0.5)">'

    for child in internal:
        attributes = child.attrib
        term = child[0]

        transform = attributes['transform']
        x = term.attrib['x']
        y = term.attrib['y']
        size = attributes['font-size']
        text = term.text
        orientation = orientations[term.text]

        fill = 'black'
        if (orientation == 'positive'):
            fill = 'rgb(44, 160, 44)'
        elif (orientation == 'negative'):
            fill = 'rgb(214, 39, 40)'

        id_tag = ids[term.text]
        new_tag = '<g transform="' + transform + '" font-size="' + str(size) + '" fill="' + fill + '" onclick="highlight(`' + id_tag + '`, `' + orientation + '`)' +'">' + '<text x="' + x + '" y="' + y + '">' + term.text + '</text></g>'
        result = result + new_tag
    result = result + '</g></svg>'
    return result

# RUN SCRIPT

In [8]:
data = pd.read_csv("fitbit_highlights.csv", sep="\t")
result_list = []
filename = "test_cloud"

for index, row in data.iterrows():
    # SETUP
    shap_data = row['Shap_json']
    highlights = row['Highlights']
    s = json.loads(shap_data)
    
    # CREATE INPUT FILE
    factors = getFactors(s)
    createInputFile(factors, filename)
    
    # CREATE CLOUDS
    generateSVG(filename)
    orientations = getOrientation(s)
    ids = getIDs(s)
    result = parseSVG(filename, ids, orientations)
    result_list.append(result)

# CREATE NEW DATAFRAM & CREATE NEW FILE
dataframe = data[["Iterator","Name", "Title", "Review", "Rating", "Highlights"]].copy()
dataframe["Word Cloud"] = result_list
dataframe

,Iterator,Name,Title,Review,Rating,Highlights,Word Cloud
0,0,Amazon Customer,Not worth the hassle,The strap is really hard to change. I managed ...,1.0,"<div class=""single""><p><img src=""avatar.jpg"" /...","<svg width=""550"" height=""300"" font-family=""Ari..."
1,1,Aubrey,Listen to the poor reviews,I've used fitbit products for years. My well l...,1.0,"<div class=""single""><p><img src=""avatar.jpg"" /...","<svg width=""550"" height=""300"" font-family=""Ari..."
2,2,Amazon Customer,It stopped working after 3 months,My fitbit stopped working after 3 months. The ...,2.0,"<div class=""single""><p><img src=""avatar.jpg"" /...","<svg width=""550"" height=""300"" font-family=""Ari..."
3,3,Sophia,scratched and not that great,I have had this watch for less than a month an...,3.0,"<div class=""single""><p><img src=""avatar.jpg"" /...","<svg width=""550"" height=""300"" font-family=""Ari..."
4,4,Cassandra Reising,All in all a good watch,I have had this for about a month and am very ...,4.0,"<div class=""single""><p><img src=""avatar.jpg"" /...","<svg width=""550"" height=""300"" font-family=""Ari..."
5,5,Gearhead,Works as designed.,Bought this for my girlfriend. Thought I was t...,4.0,"<div class=""single""><p><img src=""avatar.jpg"" /...","<svg width=""550"" height=""300"" font-family=""Ari..."
6,6,speedemonj,Great for fitness,"Great for fitness, sleep, etc but you have to ...",4.0,"<div class=""single""><p><img src=""avatar.jpg"" /...","<svg width=""550"" height=""300"" font-family=""Ari..."
7,7,Tyler,Good bang for your buck,I had a Fitbit blaze before I got the versa 2 ...,4.0,"<div class=""single""><p><img src=""avatar.jpg"" /...","<svg width=""550"" height=""300"" font-family=""Ari..."
8,8,Kindle Customer,HUGE improvement over the first Fitbit Versa,My first Fitbit Versa was great and died 4 mon...,5.0,"<div class=""single""><p><img src=""avatar.jpg"" /...","<svg width=""550"" height=""300"" font-family=""Ari..."
9,9,Nicole Quigley,Love It!!,After buying a cheaper type of fitness tracker...,5.0,"<div class=""single""><p><img src=""avatar.jpg"" /...","<svg width=""550"" height=""300"" font-family=""Ari..."
